# MMD自動トレースキット準備

このノートブックでは、MMD自動トレースの準備と実行を行います。


画面左上の「＞」をクリックして下さい。目次が開きます。

![目次](https://drive.google.com/uc?export=view&id=1x8AdFNmsIQPrtYptBf_NXPRNBJF8ON8z)


ノートブックを上から順に確認し、以下手順をひとつずつ実行してください。

- **「ランタイムをGPUに変更」**
  - ランタイムがGPUに変更できたことを確認します
  - 変更のやり方は、導入編をご確認ください
- **「Googleドライブとの連携」**
  - Googleドライブとの連携ができたことを確認します
  - 連携のやり方は、導入編をご確認ください
- **「準備開始」**
    - 準備セクションのセルをすべて実行します
      - この処理で、MMD自動トレースに必要なプログラムやデータがすべてcolab上に作成されます。
      - 大体40～60分くらいかかります。
- **「MMD自動トレースキット実行」**
  - 実行セクションのセルを上から順番に1つずつ実行します
    - トレース元動画の指定
    - トレースパラメーターの設定
    - トレース処理実行
    - 人数にもよりますが、6000Fで大体50～60分くらいかかります。


## ランタイムをGPUに変更

ヘッダの "ランタイム"　＞　"ランタイムのタイプを変更"　＞　"GPU"　を選択して下さい。

変更できたら、下のセルを実行して下さい。

In [0]:
! nvcc --version
! nvidia-smi

**【OK】**

下記のように表示されていたら成功です。

![GPU変更成功](https://drive.google.com/uc?export=view&id=17CG697kiTLkwVOdH1wg2W3MSB-hyi9u5)

---

**【NG】**

下記のように表示されていたら、ランタイムの変更に失敗しているので、導入編を再度確認して、ランタイムを変更してください。

![GPU切り替え失敗](https://drive.google.com/uc?export=view&id=1tufSuT7ocWxv3HkrmA5kwlemhu0gv6Je)

## Googleドライブとの連携

Googleドライブの `autotrace` フォルダと連携します。

下のセルを実行してください。

In [0]:
from google.colab import drive
import os

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
base_path = "/gdrive/My Drive/autotrace"

! echo "autotraceフォルダの中身 -----------"
! ls -l "$base_path"
! echo "--------------------"

## 準備開始

ここでは、準備セクションのセルを一括で実行します。

目次の「MMD自動トレースキット実行」を選択して下さい。

![実行選択](https://drive.google.com/uc?export=view&id=1YaSrtio06mmu94RUkU5dSolNtFqPiAnV)

ヘッダの　"ランタイム"　＞　"より前のセルを実行"　を選択すると、準備セクションのすべてのセルが順次実行されます。

![すべてのセルを実行](https://drive.google.com/uc?export=view&id=1rIbVI_Qyjs8idEzkIR0IBIx6Dbp3te08)

---

** 【OK】**

画面の一番下に、以下のように出力されれば、完了です。

![処理成功](https://drive.google.com/uc?export=view&id=1D21xezv6QN0RQF5ZU_LR7PRnOk0Dw4Sc)

大体40～60分くらいかかります。

---

**【NG】**

![処理失敗](https://drive.google.com/uc?export=view&id=1t-immeF3Ji1_GBNatZOG1C07j42de4Rq)

最後の行に、「No such file or directory」と出力されていたら失敗です。

解決方法が分からない場合、ノートブックを共有してください。

## 環境設定

In [0]:
# 処理時間計測のための開始時間
import time
start_time = time.time()

In [0]:
# Openposeバージョンタグ
ver_openpose = "v1.4.0"

In [0]:
# MMD自動トレースキットバージョンタグ
ver_tag = "ver1.00"

## cmake

In [0]:
! wget -c "https://github.com/Kitware/CMake/releases/download/v3.13.4/cmake-3.13.4.tar.gz"
! tar xf cmake-3.13.4.tar.gz
! cd cmake-3.13.4 && ./configure && make && sudo make install

## Openpose

In [0]:
# ライブラリのインストール

# Basic
! sudo apt-get --assume-yes update
! sudo apt-get --assume-yes install build-essential
# OpenCV
! sudo apt-get --assume-yes install libopencv-dev
# General dependencies
! sudo apt-get --assume-yes install libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
! sudo apt-get --assume-yes install --no-install-recommends libboost-all-dev
# Remaining dependencies, 14.04
! sudo apt-get --assume-yes install libgflags-dev libgoogle-glog-dev liblmdb-dev
# Python2 libs
! sudo apt-get --assume-yes install python-setuptools python-dev build-essential
! sudo easy_install pip
! sudo -H pip install --upgrade numpy protobuf opencv-python
# Python3 libs
! sudo apt-get --assume-yes install python3-setuptools python3-dev build-essential
! sudo apt-get --assume-yes install python3-pip
! sudo -H pip3 install --upgrade numpy protobuf opencv-python
# OpenCV 2.4 -> Added as option
# # sudo apt-get --assume-yes install libopencv-dev
# OpenCL Generic
! sudo apt-get --assume-yes install opencl-headers ocl-icd-opencl-dev
! sudo apt-get --assume-yes install libviennacl-dev

In [0]:
#  Openpose の clone
#! git clone  --depth 1 -b "$ver_openpose" https://github.com/CMU-Perceptual-Computing-Lab/openpose.git 
! git clone  --depth 1 https://github.com/CMU-Perceptual-Computing-Lab/openpose.git     

In [0]:
#  Openpose の モデルデータDL
! cd openpose/models && ./getModels.sh

In [0]:
# Openpose の ビルド
! sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
! cd openpose && rm -r build || true && mkdir build && cd build && cmake .. && make -j`nproc` # example demo usage

## FCRN-DepthPrediction-vmd

In [0]:
# FCRN-DepthPrediction-vmd の clone
# FIXME ブランチ指定
! git clone  --depth 1 -b "20190220" https://github.com/miu200521358/FCRN-DepthPrediction-vmd.git

In [0]:
# FCRN-DepthPrediction-vmd の モデルデータDL

# モデルデータ の導入
! mkdir -p ./FCRN-DepthPrediction-vmd/tensorflow/data

# モデルデータのダウンロード
! cd  ./FCRN-DepthPrediction-vmd/tensorflow/data && wget -c "http://campar.in.tum.de/files/rupprecht/depthpred/NYU_FCRN-checkpoint.zip" && unzip NYU_FCRN-checkpoint.zip

## 3d-pose-baseline-vmd

In [0]:
# 3d-pose-baseline-vmd の clone
! git clone  --depth 1 -b "$ver_tag" https://github.com/miu200521358/3d-pose-baseline-vmd.git

In [0]:
# 3d-pose-baseline-vmd の Human3.6MモデルデータDL

# Human3.6Mモデルデータ の導入
! mkdir -p ./3d-pose-baseline-vmd/data

# Human3.6Mモデルデータのダウンロード
! cd  ./3d-pose-baseline-vmd/data && wget -c "https://www.dropbox.com/s/e35qv3n6zlkouki/h36m.zip" && unzip h36m.zip

In [0]:
# 3d-pose-baseline-vmd の 学習データDL

# 3d-pose-baseline用学習データ の導入
! mkdir -p ./3d-pose-baseline-vmd/experiments

# 3d-pose-baseline用学習データのダウンロード
file_id = "1v7ccpms3ZR8ExWWwVfcSpjMsGscDYH7_"
file_name = "experiments.zip"
! cd  ./3d-pose-baseline-vmd && curl -sc ./cookie "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
code = "$(awk '/_warning_/ {print $NF}' ./cookie)"  
! cd  ./3d-pose-baseline-vmd && curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$code&id=$file_id" -o "$file_name"
! cd  ./3d-pose-baseline-vmd && unzip experiments.zip

## VMD-3d-pose-baseline-multi

In [0]:
# VMD-3d-pose-baseline-multi の clone

! git clone  --depth 1 -b "$ver_tag" https://github.com/miu200521358/VMD-3d-pose-baseline-multi.git

In [0]:
# VMD-3d-pose-baseline-multi のライブラリ

! sudo apt-get install python3-pyqt5  
! sudo apt-get install pyqt5-dev-tools
! sudo apt-get install qttools5-dev-tools

## 準備結果確認

In [0]:
# サンプルの実行確認
! cd openpose && ./build/examples/openpose/openpose.bin --video examples/media/video.avi --write_json ./output/ --display 0  --write_video ./output/openpose.avi

In [0]:
import time

elapsed_time = (time.time() - start_time) / 60

! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
! echo "■■すべての処理が終了しました"
! echo "■■"
! echo "■■処理にかかった時間：" "$elapsed_time" "分"
! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

! echo "Openpose実行結果"

! ls -l ./openpose/output/openpose.avi

# MMD自動トレースキット実行

ここからは、キットを実際に実行していきます。
まずは、下のセルを実行して、インストールがすべて完了しているか確認してください。



In [0]:
!ls -l ./openpose/README.md
!ls -l ./FCRN-DepthPrediction-vmd/README.md
!ls -l ./3d-pose-baseline-vmd/README.md
!ls -l ./VMD-3d-pose-baseline-multi/README.md

**【OK】**

以下のように、ファイル名とファイルサイズが表示されていれば、インストール完了です。
入力映像ファイルアップロードに進んでください。

![インストール成功](https://drive.google.com/uc?export=view&id=1l13A2iF9oTpGcZSe9q8k7JyDyiABxOQT)

---

**【NG】**

以下のように、「No such file or directory」と表示されている場合、インストール失敗です。

![インストール失敗](https://drive.google.com/uc?export=view&id=1LuKoSMwFOzFg8NguFxqAtmQy9B1_KMXr)

このノートブックの先頭に戻って初めから実行しなおいてください。

３回やってもインストールに失敗する場合、ノートブックを共有してください。

## 入力映像ファイルアップロード

処理したい映像ファイルを、準備してください。

 - ファイル名は **半角英数字のみ** にしてください。
 - Googleドライブの **autotrace** フォルダ 直下に置いてください。
 - FPSは、**30fps** もしくは **60fps** にしてください。
 - 大きさは、**1280x720** にしてください。
 - 大きさもしくはFPSが指定通りではない場合、プログラム側で再エンコードします。（fpsは30になります）
 - **マウント後のGooleドライブ上のファイルの上書きや更新は正しく認識されません。** 新しいファイルは新規の名前でアップロードしてから処理して下さい。
 - 複数人数のトレースの場合、画面上から人物が消えると、並び順が取得できなくなるので、できるだけ取得したい人数分全員が映っているようにしてください。
 - アップロードが完了したら、下のセルを順次実行して下さい。

In [0]:
#@markdown ### 【O】入力映像ファイル
#@markdown 解析対象となる映像のファイルの名前を入力して下さい。
#@markdown 横幅が1280ではない、もしくは30fpsではない場合、再エンコードします。
input_video_name = "input.mp4"  #@param {type: "string"}

from google.colab import drive
import os
import cv2
import datetime

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
base_path = "/gdrive/My Drive/autotrace"

! echo "autotraceフォルダの中身 -----------"
! ls -l "$base_path"
! echo "--------------------"

# 入力動画ファイル
input_video = base_path + "/"+ input_video_name

print("ファイル名: ", os.path.basename(input_video))
print("ファイルサイズ: ", os.path.getsize(input_video))


video = cv2.VideoCapture(input_video)
# 幅
W = video.get(cv2.CAP_PROP_FRAME_WIDTH)
# 高さ
H = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
# 総フレーム数
count = video.get(cv2.CAP_PROP_FRAME_COUNT)
# fps
fps = video.get(cv2.CAP_PROP_FPS)

print("横: {0}, 縦: {1}, フレーム数: {2}, fps: {3}".format(W, H, count, fps))



width = 1280
height = 720

if W != 1280 or (fps != 30 and fps != 60):
    print("大きさもしくはfpsが処理対象外のため、再エンコードします: "+ input_video)
    
    # 縮尺
    scale = width / W
    
    # 高さ
    height = int(H * scale)

    # 出力ファイルパス
    out_name = 'recode_{0}.avi'.format("{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now()))
    out_path = '{0}/{1}'.format(base_path, out_name)
    
    try:
        fourcc = cv2.VideoWriter_fourcc(*"I420")
        out = cv2.VideoWriter(out_path, fourcc, 30.0, (width, height), True)
        # 入力ファイル
        cap = cv2.VideoCapture(input_video)

        while(cap.isOpened()):
            # 動画から1枚キャプチャして読み込む
            flag, frame = cap.read()  # Capture frame-by-frame

            # 動画が終わっていたら終了
            if flag == False:
                break

            # 縮小
            output_frame = cv2.resize(frame, (width, height))

            # 出力
            out.write(output_frame)

        # 終わったら開放
        out.release()
    except Exception as e:
        print("再エンコード失敗", e)

    cap.release()
    cv2.destroyAllWindows()
    
    print('MMD入力用AVIファイル再生成', out_path)
    input_video_name = out_name

    # 入力動画ファイル再設定
    input_video = base_path + "/"+ input_video_name
    
    video = cv2.VideoCapture(input_video)
    # 幅
    W = video.get(cv2.CAP_PROP_FRAME_WIDTH)
    # 高さ
    H = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
    # 総フレーム数
    count = video.get(cv2.CAP_PROP_FRAME_COUNT)
    # fps
    fps = video.get(cv2.CAP_PROP_FPS)

    print("【再チェック】横: {0}, 縦: {1}, フレーム数: {2}, fps: {3}".format(W, H, count, fps))

    
!echo "入力映像ファイルは" "$input_video_name" "です。"
!echo ""
!echo "問題なければ、次へ進んで下さい。"

最後のファイル名が取得できていたら成功です。

---
**【OK】**

ファイルの大きさとfpsが指定通りの場合、入力ファイルをそのまま扱います。

![OK](https://drive.google.com/uc?export=view&id=1lvOhCAj99_NUNDb-wfRAxeu-o0Exth7v)

----
**【再エンコード】**

ファイルの大きさとfpsが指定通りではない場合、再エンコードしたaviファイルを入力ファイルとして扱います。

![再エンコード](https://drive.google.com/uc?export=view&id=1xEiy-pdeHWQpt4CLZePg9YT1_7U8bEqz)


パラメーターを設定して、自動トレースを実行して下さい。

 - 【O】… Openposeで使用するパラメーター
 - 【F】… FCRN-DepthPrediction-vmdで使用するパラメーター
 - 【V】… VMD-3d-pose-baseline-multiで使用するパラメーター

## パラメーター設定

In [0]:
#@markdown 映像をトレースする際のパラメーターを入力して、セルを実行して下さい。

#@markdown --- 

#@markdown ### 【O】映像に映っている最大人数
#@markdown 映像から取得したい人数を入力して下さい。
#@markdown 可能な限りこの人数分映っているように、映像データを加工しておいてください。
number_people_max = 1  #@param {type: "number"}

#@markdown --- 

#@markdown ### 【O】解析を開始するフレーム
#@markdown 解析を開始するフレームNoを入力して下さい。(0始まり)
#@markdown 最初にロゴが表示されている等、人体が正確にトレースできない場合に、全員が映像内に映っている最初のフレームを指定してください。
frame_first = 0  #@param {type: "number"}

#@markdown ---

#@markdown ### 【F】反転フレームリスト
#@markdown Openposeが誤認識して反転しているフレーム番号(0始まり)を指定してください。
#@markdown ここで指定された番号のフレームに対して、反転判定を行い、反転認定された場合、関節位置が反転されます。
#@markdown カンマで複数件指定可能です。また、ハイフンで範囲が指定可能です。
#@markdown 例）4,10-12　…　4,10,11,12 が反転判定対象フレームとなります。
#@markdown デフォルトで、全フレームが反転判定対象となるよう設定しています。
#@markdown 不要な回転等があった場合には、そのフレームをリストから除いて指定すると、回転がなくなる可能性があります。
#@markdown 例）0-1112,1120-10000　…　1113-1119Fの間だけ、反転判定対象から除きます
reverse_frames = "0-100000"  #@param {type: "string"}

#@markdown --- 

#@markdown ### 【F】順番指定リスト
#@markdown 複数人数トレースで、交差後の人物INDEX順番を指定してください。1人トレースの場合は空欄のままで大丈夫です。
#@markdown Openposeが0F目で認識した順番に0, 1, とINDEXが割り当てられます。
#@markdown フォーマット：［＜フレーム番号＞:0番目に推定された人物のインデックス,1番目に推定された人物のインデックス, …］
#@markdown 例）[10:1,0]　…　10F目は、左から1番目の人物、0番目の人物の順番に並べ替えます。
#@markdown message.logに上記フォーマットで、どのような順番で出力したかを残しているので、それを参考にしてください。
#@markdown [10:1,0][30:0,1]のように、カッコ単位で複数件指定可能です。
#@markdown また、output_XXX.aviでは、推定された順番に人物に色が割り当てられています。体の右半分は赤、左半分は以下の色になります。
#@markdown 0:緑, 1:青, 2:白, 3:黄, 4:桃,  5:水色, 6:濃緑, 7:濃青, 8:灰色, 9:濃黄, 10:濃桃, 11:濃水色
order_specific = ""  #@param {type: "string"}

#@markdown --- 

#@markdown ### 【V】ボーン構造CSVファイル
#@markdown トレース対象モデルのボーン構造CSVファイルのパスを選択もしくは入力して下さい。
#@markdown あにまさ式ミクと、あにまさ式ミク準標準が選べる他、任意のモデルのボーン構造CSVファイルが入力可能です。
#@markdown 任意のモデルボーン構造CSVファイルを入力する場合、Googleドライブの "autotrace" フォルダにcsvファイルをアップロードしてください。
#@markdown そして「/gdrive/My Drive/autotrace/[csvファイル名]」のように入力して下さい。
born_model_csv = "born/\u3042\u306B\u307E\u3055\u5F0F\u30DF\u30AF\u30DC\u30FC\u30F3.csv" #@param ["born/\u3042\u306B\u307E\u3055\u5F0F\u30DF\u30AF\u30DC\u30FC\u30F3.csv", "born/\u3042\u306B\u307E\u3055\u5F0F\u30DF\u30AF\u6E96\u6A19\u6E96\u30DC\u30FC\u30F3.csv"] {allow-input: true}


#@markdown --- 

#@markdown ### 【V】IKで出力するか
#@markdown 足をIKで出力するか、yes か no を選んで下さい。
#@markdown no を入力した場合、FKで出力します
ik_flag = "yes"  #@param ['yes', 'no']
is_ik = 1 if ik_flag == "yes" else 0

#@markdown ---

#@markdown ### 【V】踵位置補正
#@markdown 踵のY軸補正値を数値(小数可)で入力して下さい。
#@markdown マイナス値を入力すると地面に近付き、プラス値を入力すると地面から遠ざかります。
#@markdown ある程度は自動で補正しますが、補正しきれない場合に、設定して下さい。
heel_position = 0.0  #@param {type: "number"}

#@markdown ---

#@markdown ### 【V】センターZ移動倍率
#@markdown センターZ移動に掛ける倍率を数値(小数可)で入力して下さい。
#@markdown 値が小さいほど、センターZ移動の幅が小さくなります。
#@markdown 0を入力した場合、センターZ軸移動を行いません。
center_z_scale = 5  #@param {type: "number"}

#@markdown ---

#@markdown ### 【V】滑らかさ
#@markdown モーションの円滑化の度数を指定します
#@markdown 1以上の整数のみを入力して下さい。
#@markdown 度数が大きいほど、円滑化されます。（代わりに動作が小さくなります）
smooth_times = 1  #@param {type: "number"}

#@markdown ---

#@markdown ### 【V】移動間引き量
#@markdown 移動キー（IK・センター）の間引きに使用する移動量を数値(小数可)で指定します
#@markdown 指定された範囲内の移動があった場合に間引きされます。
#@markdown 移動間引き量を0にした場合、間引きを行いません。
threshold_pos = 0.5  #@param {type: "number"}

#@markdown ---

#@markdown ### 【V】回転間引き角度
#@markdown 回転キーの間引きに使用する角度(0～180度まで小数可)を指定します
#@markdown 指定された角度以内の回転があった場合に間引きされます。
threshold_rot = 3  #@param {type: "number"}



!echo 映像に映っている最大人数: "$number_people_max"
!echo 解析を開始するフレーム: "$frame_first"
!echo 反転フレームリスト: "$reverse_frames"
!echo 順番指定リスト: "$order_specific"
!echo ボーン構造CSVファイル: "$born_model_csv"
!echo IKで出力するか: "$ik_flag"
!echo 踵位置補正: "$heel_position"
!echo センターZ移動倍率: "$center_z_scale"
!echo 滑らかさ: "$smooth_times"
!echo 移動間引き量: "$threshold_pos"
!echo 回転間引き角度: "$threshold_rot"

!echo ""
!echo 上記で間違いない場合、次に進んで下さい。

## 自動トレース実行（全実行）

In [0]:
#@markdown フォームの入力がすべて完了したら、このセルを実行してください。
#@markdown 以下の順番で処理が実行されます。

#@markdown 1. Openpose（映像→2D）
#@markdown 2. FCRN-DepthPrediction-vmd（深度推定）
#@markdown 3. 3d-pose-baseline-vmd（2D→3D）
#@markdown 4. VMD-3d-pose-baseline-multi（3D→VMD）

#@markdown トレース人数にもよりますが、6000Fで大体50～60分くらいかかります
#@markdown Openposeが開始すると、しばらく細長い四角が出たまま動かなくなったように見えます。
#@markdown 背後で処理は行われていますので、何も操作せずお待ちください。
#@markdown vmdファイルが生成されていない、pos.txtの中身が空、error.txtだけがある、といった場合は、まずerror.txtの中身を確認して、「エラーが起きた場合」セクションを確認・実行して下さい。

import time
import datetime
import cv2
from google.colab import drive

start_time = time.time()

# 出力フォルダ削除
if os.path.exists("./output"):
    !rm -r ./output

# 処理日時
now_str = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
drive_base_dir = "/gdrive/My Drive/autotrace"

output_json = "/content/output/json"
output_openpose_avi = "/content/output/openpose.avi"
! mkdir -p "$output_json"

# 出力用Googleドライブフォルダ
drive_dir_path = drive_base_dir + "/" + now_str 
! mkdir -p "$drive_dir_path"

! echo ------------------------------------------
! echo Openpose
! echo ------------------------------------------

# Openpose実行
! cd openpose/ && ./build/examples/openpose/openpose.bin --video "$input_video" --display 0 --model_pose COCO --write_json "$output_json" --write_video "$output_openpose_avi" --frame_first "$frame_first" --number_people_max "$number_people_max"

! echo ------------------------------------------
! echo FCRN-DepthPrediction-vmd
! echo ------------------------------------------

! cd FCRN-DepthPrediction-vmd && python tensorflow/predict_video.py --model_path tensorflow/data/NYU_FCRN.ckpt --video_path "$input_video" --json_path "$output_json" --interval 10 --reverse_frames "$reverse_frames" --order_specific "$order_specific" --verbose 1 --now "$now_str" --avi_output "yes"  --number_people_max "$number_people_max"
    
# 深度結果コピー
depth_dir_path =  output_json + "_" + now_str + "_depth"

if os.path.exists( depth_dir_path + "/error.txt"):
    
    # エラー発生
    ! cp "$depth_dir_path"/error.txt "$drive_dir_path"

    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
    ! echo "■■エラーが発生したため、処理を中断しました。"
    ! echo "■■"
    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

    ! echo "$drive_dir_path" "の error.txt の中身を確認してください。"

else:
    
    ! cp "$depth_dir_path"/*.avi "$drive_dir_path"
    ! cp "$depth_dir_path"/message.log "$drive_dir_path"
    ! cp "$depth_dir_path"/reverse_frames.txt "$drive_dir_path"
    ! cp "$depth_dir_path"/order_specific.txt "$drive_dir_path"

    for i in range(1, number_people_max+1):
        ! echo ------------------------------------------
        ! echo 3d-pose-baseline-vmd ["$i"]
        ! echo ------------------------------------------

        target_name = "_" + now_str + "_idx0" + str(i)
        target_dir = output_json + target_name

        !cd ./3d-pose-baseline-vmd && python src/openpose_3dpose_sandbox_vmd.py --camera_frame --residual --batch_norm --dropout 0.5 --max_norm --evaluateActionWise --use_sh --epochs 200 --load 4874200 --gif_fps 30 --verbose 1 --openpose "$target_dir" --person_idx 1    

        ! echo ------------------------------------------
        ! echo VMD-3d-pose-baseline-multi ["$i"]
        ! echo ------------------------------------------

        ! cd ./VMD-3d-pose-baseline-multi && python applications/pos2vmd_multi.py -v 2 -t "$target_dir" -b "$born_model_csv" -c 30 -z "$center_z_scale" -s "$smooth_times" -p "$threshold_pos" -r "$threshold_rot" -k "$is_ik" -e "$heel_position"

        # INDEX別結果コピー
        idx_dir_path = drive_dir_path + "/idx0" + str(i)
        ! mkdir -p "$idx_dir_path"
        ! cp "$target_dir"/*.vmd "$idx_dir_path"
        ! cp "$target_dir"/pos.txt "$idx_dir_path"

    # Googleドライブ再マウント
    drive.mount('/gdrive')

    elapsed_time = (time.time() - start_time) / 60

    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
    ! echo "■■すべての処理が終了しました"
    ! echo "■■"
    ! echo "■■処理にかかった時間：" "$elapsed_time" "分"
    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

    ! echo ""
    ! echo "MMD自動トレース実行結果"

    ! echo "$drive_dir_path"
    ! ls -l "$drive_dir_path"

## 自動トレース再実行(深度推定以降)

In [0]:
#@markdown Openposeの処理が終わっていて、【F】【V】のパラメーターを調整して再実行したい場合

#@markdown 1. 「パラメーター設定」の【F】【V】のパラメーターを埋める
#@markdown 2. 「パラメーター設定」のセルを実行する
#@markdown 3. このセルを実行する

#@markdown 新しいパラメーターで、FCRN-DepthPrediction-vmd（深度推定）以降のみが実行されます。
#@markdown トレース元動画や【O】のパラメーターを変えたい場合、「入力映像ファイルアップロード」からすべて再実行してください。

import time
import datetime
import cv2
import os
from google.colab import drive

start_time = time.time()

# 処理日時
now_str = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
drive_base_dir = "/gdrive/My Drive/autotrace"

output_json = "/content/output/json"
output_openpose_avi = "/content/output/openpose.avi"

# 出力用Googleドライブフォルダ
drive_dir_path = drive_base_dir + "/" + now_str 
! mkdir -p "$drive_dir_path"

! echo ------------------------------------------
! echo FCRN-DepthPrediction-vmd
! echo ------------------------------------------

! cd FCRN-DepthPrediction-vmd && python tensorflow/predict_video.py --model_path tensorflow/data/NYU_FCRN.ckpt --video_path "$input_video" --json_path "$output_json" --interval 10 --reverse_frames "$reverse_frames" --order_specific "$order_specific" --verbose 1 --now "$now_str" --avi_output "yes"  --number_people_max "$number_people_max"
    
# 深度結果コピー
depth_dir_path =  output_json + "_" + now_str + "_depth"

if os.path.exists( depth_dir_path + "/error.txt"):
    
    # エラー発生
    ! cp "$depth_dir_path"/error.txt "$drive_dir_path"

    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
    ! echo "■■エラーが発生したため、処理を中断しました。"
    ! echo "■■"
    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

    ! echo "$drive_dir_path" "の error.txt の中身を確認してください。"

else:
    
    ! cp "$depth_dir_path"/*.avi "$drive_dir_path"
    ! cp "$depth_dir_path"/message.log "$drive_dir_path"
    ! cp "$depth_dir_path"/reverse_frames.txt "$drive_dir_path"
    ! cp "$depth_dir_path"/order_specific.txt "$drive_dir_path"

    for i in range(1, number_people_max+1):
        ! echo ------------------------------------------
        ! echo 3d-pose-baseline-vmd ["$i"]
        ! echo ------------------------------------------

        target_name = "_" + now_str + "_idx0" + str(i)
        target_dir = output_json + target_name

        !cd ./3d-pose-baseline-vmd && python src/openpose_3dpose_sandbox_vmd.py --camera_frame --residual --batch_norm --dropout 0.5 --max_norm --evaluateActionWise --use_sh --epochs 200 --load 4874200 --gif_fps 30 --verbose 1 --openpose "$target_dir" --person_idx 1    

        ! echo ------------------------------------------
        ! echo VMD-3d-pose-baseline-multi ["$i"]
        ! echo ------------------------------------------

        ! cd ./VMD-3d-pose-baseline-multi && python applications/pos2vmd_multi.py -v 2 -t "$target_dir" -b "$born_model_csv" -c 30 -z "$center_z_scale" -s "$smooth_times" -p "$threshold_pos" -r "$threshold_rot" -k "$is_ik" -e "$heel_position"

        # INDEX別結果コピー
        idx_dir_path = drive_dir_path + "/idx0" + str(i)
        ! mkdir -p "$idx_dir_path"
        ! cp "$target_dir"/*.vmd "$idx_dir_path"
        ! cp "$target_dir"/pos.txt "$idx_dir_path"

    # Googleドライブ再マウント
    drive.mount('/gdrive')

    elapsed_time = (time.time() - start_time) / 60

    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
    ! echo "■■すべての処理が終了しました"
    ! echo "■■"
    ! echo "■■処理にかかった時間：" "$elapsed_time" "分"
    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

    ! echo ""
    ! echo "MMD自動トレース実行結果"

    ! echo "$drive_dir_path"    
    ! ls -l "$drive_dir_path"

## 自動トレース再実行(VMDのみ)

In [0]:
#@markdown 【V】（VMDのパラメーター）だけを調整して再実行したい場合

#@markdown 1. 「パラメーター設定」の【V】のパラメーターを埋める
#@markdown 2. 「パラメーター設定」のセルを実行する
#@markdown 3. このセルの「INDEX番号」欄を入力する
#@markdown 4. このセルを実行する

#@markdown 新しいパラメーターと指定されたINDEXで、VMD-3d-pose-baseline-multi（3D→VMD）のみが実行されます。
#@markdown **Googleドライブに出力されたidxフォルダを再利用しますので、削除しないでください。**
#@markdown ---

#@markdown ### INDEX番号
#@markdown 処理を実行したいINDEXの番号のみを入力して下さい。
i = 1  #@param {type: "number"}

import time
import datetime
import cv2
from google.colab import drive

start_time = time.time()

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
drive_base_dir = "/gdrive/My Drive/autotrace"

output_json = "/content/output/json"
output_openpose_avi = "/content/output/openpose.avi"

# 出力用Googleドライブフォルダ
drive_dir_path = drive_base_dir + "/" + now_str 
! mkdir -p "$drive_dir_path"

target_name = "_" + now_str + "_idx0" + str(i)
target_dir = output_json + target_name

! echo ------------------------------------------
! echo VMD-3d-pose-baseline-multi ["$i"]
! echo ------------------------------------------

! cd ./VMD-3d-pose-baseline-multi && python applications/pos2vmd_multi.py -v 2 -t "$target_dir" -b "$born_model_csv" -c 30 -z "$center_z_scale" -s "$smooth_times" -p "$threshold_pos" -r "$threshold_rot" -k "$is_ik" -e "$heel_position"

# INDEX別結果コピー
idx_dir_path = drive_dir_path + "/idx0" + str(i)
! mkdir -p "$idx_dir_path"
! cp "$target_dir"/*.vmd "$idx_dir_path"
! cp "$target_dir"/pos.txt "$idx_dir_path"

# Googleドライブ再マウント
drive.mount('/gdrive')
    
elapsed_time = (time.time() - start_time) / 60
    
! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
! echo "■■すべての処理が終了しました"
! echo "■■"
! echo "■■処理にかかった時間：" "$elapsed_time" "分"
! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

! echo ""
! echo "MMD自動トレース実行結果"

! echo "$idx_dir_path"
! ls -l "$idx_dir_path"/

# エラーが起きた場合

エラーが起きた場合、vmdファイルが生成されていない場合は、このセクションを上からひとつずつ実行してください。

それでも解決しない場合、導入編の手順に従って、ノートブックのコピーを共有してください。

## 1. Openposeが読み取る最初のフレームに、人数分映っていない場合

error.txtに「最初のフレームに人数分のデータがありません。」と記載されている場合、Openposeが読み取る最初のフレームに人数分のデータがない事が原因です。

下のセルを実行してください。

In [0]:
!find output/json/ -name "*.json" | sort | head -n 1 | xargs ls -l
!find output/json/ -name "*.json" | sort | head -n 1 | xargs cat

【状況】
下記のように、peopleの後ろにデータがない場合、0F目に人物データが取得できていません。

![結果なし](https://drive.google.com/uc?export=view&id=1osssF0NCWply6J0-zPIhN2wm1gT0o6Io)

【解決方法】

人物が映っている最初のフレームを「【O】解析を開始するフレーム」に指定してください。

「Openposeが読み取ったフレームリスト30件」のセルを実行すると、先頭30件のOpenpose結果ファイルが表示されます。

![先頭30件](https://drive.google.com/uc?export=view&id=1lxP78w4NIbQSKWhpfbynCjDKOgmcNp0o)

人物データがないJSONファイルは、ファイルサイズがとても小さいです。（図の場合、0F目が人物データなし）

1人分のデータで大体500Byte前後(0.5KB)のファイルサイズになります。

これを参考にして、先頭のフレーム番号を決めてください。元動画の編集や再アップロードは不要です。

先頭のフレーム番号が決まったら、「パラメーター設定」セクションの「【O】解析を開始するフレーム」に、その番号を入力して、「パラメーター設定」＞「自動トレース実行（全実行）」の順で実行してください。


複数人トレースの場合は、0F目（【O】解析を開始するフレーム）には全員映っている必要があります。
ファイルサイズも人数分増えますので、目安にしてください。

## 2. Openposeが読み取ったフレームリスト先頭30件

In [0]:
ls -l output/json/*.json | head -n 30

## 3. Googleドライブにファイルが追加されない場合

error.txt もvmdも何も出力されなかった場合、セルの出力結果を確認してください。

最後にファイル名のリストが出ている場合、出力自体は成功しています。

ただ、Googleドライブとの連携が済んでいるにも関わらず、データが反映されないケースを確認しています。

詳細は調査中ですが、とりあえずの対応として、クラウド上の元データをダウンロードしてください。

1. 目次の横にある「ファイル」欄をクリックする
2. ヘッダの「更新」をクリックする
3. output ＞ json を開く
4. xxx_depth ＞ output_XXX.avi …　背景AVI(MMD)
5. xxx_idxXX ＞ output_XXX.vmd　…　モーションデータ(MMD)
6. xxx_idxXX ＞ pos.txt　…　3D関節位置データ(Unity)
7. 複数人数のトレースした場合、idxが複数件できています。

![クラウドデータ](https://drive.google.com/uc?export=view&id=1fArRyRdfs1kBLaLTpdkdJ-MYwHNe-UUq)


# TIPS

上記の他、思いつくままに、参考になりそうな事を。

## オススメの作業順番


私は以下の順番で作業を行っています。

1. 「【O】映像に映っている最大人数」に、トレース元動画からトレースしたい人数を入力して、セルを実行する
2. 「自動トレース実行（全実行）」のセルを実行する
3. 結果がエラーになった場合、「エラーが起きた場合」の「Openposeが読み取る最初のフレームに、人数分映っているか」を実行して、人物が映っているフレームを確認する。成功していたら6に移動。
4. 「【O】解析を開始するフレーム」に、3で見つけたフレーム番号を入力する
5. 「パラメーター設定」のセルを実行する
6. 「自動トレース再実行(深度推定以降)」のセルを実行する
7.  意図しない回転があった場合、「【F】反転フレームリスト」に該当フレームを除いて設定する
8. 複数人数のトレースで、入れ替わりが認識できていない場合、message.logを見ながら、「【F】順番指定リスト」に順番を指定する
9. 「パラメーター設定」のセルを実行する
10. 「自動トレース再実行(深度推定以降)」のセルを実行する
11. 納得いくまで、7～10を繰り返す
12. 回転や入れ替えの指定が完了したら、【V】のパラメーターを設定する
13. 「パラメーター設定」のセルを実行する
14. 「自動トレース再実行(VMDのみ)」のセルを実行する。複数人数のトレースの場合、人数分実行する
15. 納得いくまで、12～14を繰り返す

頑張って下さい！

## トレースしやすい動画

 - 固定カメラである
 - 関節がはっきり分かる
    - ロングスカートや和服等、関節が見えにくい人体は苦手です
    - 背景が人物と似た色である、影が濃く映っている、などの場合、トレースを間違える事が多いです
    - 手首・足首まで映っている方が、精度が高くなります
 - 最初のフレームで前向きである
   - 後ろや横を向いていると始めのデータが綺麗に取れません（一度正面を向いてくれると直る事が多いです）
 - 最初のフレームで全身の関節が判別できる
   - どこか隠れていると、それだけ精度が落ちます
 - 頭が上、足が下
   - 逆立ちやキックなどで高く足が上がっている場合、足を手と誤認識します。
